In [4]:
import wti_env

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [35]:
import pandas as pd
import numpy as np

In [99]:
def create_merged_df(contract_type):
  spot = pd.read_csv("/Users/ethanhersch/Documents/Documents/Stanford/CS238/FinalProject/CS238_Project/data/oil_data/Cushing_OK_WTI_Spot_Price_FOB_sorted.csv")
  futures = pd.read_csv(f"/Users/ethanhersch/Documents/Documents/Stanford/CS238/FinalProject/CS238_Project/data/oil_data/NYMEX_futures/Cushing_OK_Crude_Oil_Future_Contract_{contract_type}.csv")

  df = pd.merge(spot, futures, on="Date", how="inner")
  df = df.rename(columns={"Price_y": f"CL{contract_type}"})
  df = df.rename(columns={"Price_x": "spot_price"})
  df[f"ret_CL{contract_type}"] = np.log(df[f"CL{contract_type}"].shift(-1) / df[f"CL{contract_type}"])
  return df

In [100]:
create_merged_df(1)

/Users/ethanhersch/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Date,spot_price,CL1,ret_CL1
0,1986-01-02,25.56,25.56,0.015913
1,1986-01-03,26.00,25.97,0.022841
2,1986-01-06,26.53,26.57,-0.014023
3,1986-01-07,25.85,26.20,-0.010359
4,1986-01-08,25.87,25.93,-0.001158
...,...,...,...,...
9581,2024-04-01,84.54,83.71,0.017056
9582,2024-04-02,85.95,85.15,0.003283
9583,2024-04-03,86.22,85.43,0.013487
9584,2024-04-04,87.37,86.59,0.003689


In [103]:
def add_pipeline_weather_data(df, location):
  loc_df = pd.read_csv(f"/Users/ethanhersch/Documents/Documents/Stanford/CS238/FinalProject/CS238_Project/data/processed_data/incoming/{location}_incoming.csv")

  loc_df = loc_df.rename(columns={"mean_temp_K": f"{location}_temp"})
  loc_df = loc_df.rename(columns={"total_precip_mm": f"{location}_precip"})
  df = pd.merge(df, loc_df, on="Date", how="left") # use left merge to use dates from left df
  return df

In [104]:
df = create_merged_df(1)
loc_df = add_pipeline_weather_data(df, "alaska")
loc_df

/Users/ethanhersch/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Date,spot_price,CL1,ret_CL1,alaska_temp,alaska_precip
0,1986-01-02,25.56,25.56,0.015913,248.181843,2.670807
1,1986-01-03,26.00,25.97,0.022841,248.961898,3.482124
2,1986-01-06,26.53,26.57,-0.014023,258.344380,2.313171
3,1986-01-07,25.85,26.20,-0.010359,256.710557,0.916802
4,1986-01-08,25.87,25.93,-0.001158,255.486960,1.147838
...,...,...,...,...,...,...
9581,2024-04-01,84.54,83.71,0.017056,266.456916,2.907795
9582,2024-04-02,85.95,85.15,0.003283,262.338132,1.333529
9583,2024-04-03,86.22,85.43,0.013487,259.954487,0.559277
9584,2024-04-04,87.37,86.59,0.003689,263.308430,1.070032


In [105]:
locations = ["alaska", "bakken", "western_canada", "midwest_corridor", "ok_hub", "permian", "rockies", "texas_gulf"]

df = create_merged_df(1)
for loc in locations:
  print(loc)
  df = add_pipeline_weather_data(df, loc)

df

alaska
bakken
western_canada
midwest_corridor
ok_hub
permian
rockies
texas_gulf


/Users/ethanhersch/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Date,spot_price,CL1,ret_CL1,alaska_temp,alaska_precip,bakken_temp,bakken_precip,western_canada_temp,western_canada_precip,midwest_corridor_temp,midwest_corridor_precip,ok_hub_temp,ok_hub_precip,permian_temp,permian_precip,rockies_temp,rockies_precip,texas_gulf_temp,texas_gulf_precip
0,1986-01-02,25.56,25.56,0.015913,248.181843,2.670807,267.462871,4.618883,259.668224,2.426405,272.636515,1.420228,280.687172,0.000855,282.793781,0.000860,269.915554,1.112069,284.128822,0.489698
1,1986-01-03,26.00,25.97,0.022841,248.961898,3.482124,262.755624,0.840877,257.547465,0.577723,272.069060,0.977935,279.298853,0.000861,282.357327,0.001564,268.373695,1.647863,285.753964,0.373387
2,1986-01-06,26.53,26.57,-0.014023,258.344380,2.313171,254.950851,0.518769,255.220885,0.686386,265.290904,0.105560,278.176843,0.159463,281.672159,0.020884,267.812953,2.884952,280.767185,0.613318
3,1986-01-07,25.85,26.20,-0.010359,256.710557,0.916802,257.554692,0.141348,261.864550,0.173848,260.424234,0.100934,274.819601,0.444359,277.907672,5.020631,266.607039,1.334517,283.705630,3.066794
4,1986-01-08,25.87,25.93,-0.001158,255.486960,1.147838,268.006027,0.000200,268.871573,0.870102,263.865249,0.010327,271.596204,0.038997,271.975373,2.905248,266.647574,0.000956,278.790667,6.579200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9581,2024-04-01,84.54,83.71,0.017056,266.456916,2.907795,273.980103,0.103339,274.328369,0.498237,282.581676,6.058377,294.530965,3.144950,294.180991,1.217820,274.246254,3.640875,296.917074,0.293375
9582,2024-04-02,85.95,85.15,0.003283,262.338132,1.333529,277.604200,0.008144,276.605366,0.305583,280.158687,11.657735,287.310766,7.704964,287.198399,0.300033,275.216672,0.248428,295.829715,2.786975
9583,2024-04-03,86.22,85.43,0.013487,259.954487,0.559277,278.807268,0.018436,278.190696,2.470537,278.711498,2.718310,285.542092,0.003279,287.183338,0.018029,278.428109,0.030500,291.111963,0.512906
9584,2024-04-04,87.37,86.59,0.003689,263.308430,1.070032,281.382040,0.020203,274.833164,2.265301,279.489586,0.431029,287.226755,0.001722,291.200820,0.001722,280.880427,0.080914,291.564768,0.001741


In [108]:
def add_producing_weather_data(df, location):
  loc_df = pd.read_csv(f"/Users/ethanhersch/Documents/Documents/Stanford/CS238/FinalProject/CS238_Project/data/processed_data/producing_locations/{location}_producing.csv")

  loc_df = loc_df.rename(columns={"mean_temp_K": f"{location}_temp_producing"})
  loc_df = loc_df.rename(columns={"total_precip_mm": f"{location}_precip_producing"})
  df = pd.merge(df, loc_df, on="Date", how="left")
  return df

In [109]:
locations = ["bakken", "canada", "eagle", "central_kansas", "n_alaska", "nc_ok", "ne_ok", "nw_ok", "permian", "s_alaska", "saskatchewan", "se_kansas", "sw_kansas"]

for loc in locations:
  df = add_producing_weather_data(df, loc)

df

,Date,spot_price,CL1,ret_CL1,alaska_temp,alaska_precip,bakken_temp,bakken_precip,western_canada_temp,western_canada_precip,...,permian_temp_producing,permian_precip_producing,s_alaska_temp_producing,s_alaska_precip_producing,saskatchewan_temp_producing,saskatchewan_precip_producing,se_kansas_temp_producing,se_kansas_precip_producing,sw_kansas_temp_producing,sw_kansas_precip_producing
0,1986-01-02,25.56,25.56,0.015913,248.181843,2.670807,267.462871,4.618883,259.668224,2.426405,...,283.057839,0.000859,260.519455,3.656118,262.960859,4.904250,278.678435,0.000853,278.836331,0.000853
1,1986-01-03,26.00,25.97,0.022841,248.961898,3.482124,262.755624,0.840877,257.547465,0.577723,...,282.991359,0.001127,262.546395,10.708141,260.890291,0.671049,276.425976,0.000860,278.046152,0.000860
2,1986-01-06,26.53,26.57,-0.014023,258.344380,2.313171,254.950851,0.518769,255.220885,0.686386,...,281.706107,0.002240,264.872811,5.483261,251.438777,0.098123,274.176621,0.025036,274.774898,0.012616
3,1986-01-07,25.85,26.20,-0.010359,256.710557,0.916802,257.554692,0.141348,261.864550,0.173848,...,278.564829,5.292881,263.509002,6.184776,255.407243,0.074576,267.526799,0.004907,269.416412,0.078262
4,1986-01-08,25.87,25.93,-0.001158,255.486960,1.147838,268.006027,0.000200,268.871573,0.870102,...,272.177875,3.567226,263.460774,7.949282,269.156659,0.000003,267.496653,0.000000,270.957890,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9581,2024-04-01,84.54,83.71,0.017056,266.456916,2.907795,273.980103,0.103339,274.328369,0.498237,...,293.920574,0.717518,266.497493,4.679901,273.921081,0.006073,293.094960,8.825796,289.592458,0.345192
9582,2024-04-02,85.95,85.15,0.003283,262.338132,1.333529,277.604200,0.008144,276.605366,0.305583,...,286.947001,0.354299,266.712750,3.282834,276.939037,0.043138,283.022691,5.640424,281.943409,4.396784
9583,2024-04-03,86.22,85.43,0.013487,259.954487,0.559277,278.807268,0.018436,278.190696,2.470537,...,287.030849,0.036966,265.888355,0.484941,279.032995,0.024988,282.355414,0.006112,284.019176,0.000557
9584,2024-04-04,87.37,86.59,0.003689,263.308430,1.070032,281.382040,0.020203,274.833164,2.265301,...,291.214240,0.001722,266.710304,1.117722,280.015681,0.168145,282.616041,0.001722,286.410128,0.001722


In [96]:
df.head(20)

,Date,spot_price,CL1,ret_CL1,alaska_temp,alaska_precip,bakken_temp,bakken_precip,canada_temp,canada_precip,...,permian_temp_producing,permian_precip_producing,s_alaska_temp_producing,s_alaska_precip_producing,saskatchewan_temp_producing,saskatchewan_precip_producing,se_kansas_temp_producing,se_kansas_precip_producing,sw_kansas_temp_producing,sw_kansas_precip_producing
0,1986-01-02,25.56,25.56,0.015913,248.181843,2.670807,267.462871,4.618883,259.668224,2.426405,...,283.057839,0.000859,260.519455,3.656118,262.960859,4.904250,278.678435,0.000853,278.836331,0.000853
1,1986-01-03,26.00,25.97,0.022841,248.961898,3.482124,262.755624,0.840877,257.547465,0.577723,...,282.991359,0.001127,262.546395,10.708141,260.890291,0.671049,276.425976,0.000860,278.046152,0.000860
2,1986-01-06,26.53,26.57,-0.014023,258.344380,2.313171,254.950851,0.518769,255.220885,0.686386,...,281.706107,0.002240,264.872811,5.483261,251.438777,0.098123,274.176621,0.025036,274.774898,0.012616
3,1986-01-07,25.85,26.20,-0.010359,256.710557,0.916802,257.554692,0.141348,261.864550,0.173848,...,278.564829,5.292881,263.509002,6.184776,255.407243,0.074576,267.526799,0.004907,269.416412,0.078262
4,1986-01-08,25.87,25.93,-0.001158,255.486960,1.147838,268.006027,0.000200,268.871573,0.870102,...,272.177875,3.567226,263.460774,7.949282,269.156659,0.000003,267.496653,0.000000,270.957890,0.000000
5,1986-01-09,26.03,25.90,-0.004256,252.881021,0.843038,274.870243,0.012505,270.268323,0.739734,...,272.670942,0.104420,260.600720,4.242261,274.656911,0.009948,271.910843,0.000864,274.651738,0.000864
6,1986-01-10,25.65,25.79,-0.027916,253.941336,1.103574,270.749021,0.004333,270.499376,0.500149,...,275.545348,0.002140,263.079516,7.036984,270.347002,0.002203,275.371637,0.002140,276.613289,0.184582
7,1986-01-13,25.08,25.08,-0.004796,255.734440,0.988665,268.122110,0.000853,268.598690,0.047324,...,279.783935,0.000846,265.936989,8.240211,268.885176,0.000848,274.489215,0.000846,278.169985,0.000846
8,1986-01-14,24.97,24.96,0.007583,257.756647,2.526569,268.487953,0.000731,269.646737,0.091625,...,280.893929,0.000435,266.543377,9.096994,269.148610,0.000541,277.050598,0.000435,279.237260,0.000435
9,1986-01-15,25.18,25.15,-0.042231,256.876726,1.914837,268.494875,0.000855,269.354359,0.166806,...,283.344482,0.006340,264.600279,4.610327,269.572538,0.000852,276.213341,0.000852,279.370431,0.000852


In [112]:
df.to_csv("/Users/ethanhersch/Documents/Documents/Stanford/CS238/FinalProject/CS238_Project/src/wti_env_data.csv", index=False)